In [172]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import numpy as np
import h3

In [173]:


data_4 = pd.read_csv("C:/Users/freud/Tensorflow/DataFrames/all_waybill_info_meituan_0322.csv")



In [174]:

min_lat = 0
min_lon = 0
max_lat = 46000000
max_lon = 174700000

"first we will filter the data and remvoing a order area encupsuled from the main area of operation"

data_4 = data_4[
        (data_4['recipient_lat'] >= min_lat) &
        (data_4['recipient_lat'] <= max_lat) &
        (data_4['recipient_lng'] >= min_lon) &
        (data_4['recipient_lng'] <= max_lon) &
        (data_4['sender_lat'] >= min_lat) &
        (data_4['sender_lat'] <= max_lat) &
        (data_4['sender_lng'] >= min_lon) &
        (data_4['sender_lng'] <= max_lon)] 


data_4['platform_order_date'] = data_4['platform_order_time']
data_4['platform_order_date'] = pd.to_datetime(data_4['platform_order_date'], unit='s') + pd.Timedelta(hours=8) #Offset of 8 hours, because it is probably a chinese timezone

data_4 = data_4[data_4['estimate_meal_prepare_time'] != 0].reset_index(drop=True) #data cleaning
data_4 = data_4[data_4['is_prebook'] != 1].reset_index(drop=True) #data cleaning

scaling_factor = 1_000_000
data_4['sender_lat'] = data_4['sender_lat'] / scaling_factor
data_4['sender_lng'] = data_4['sender_lng'] / scaling_factor
data_4['recipient_lat'] = data_4['recipient_lat'] / scaling_factor
data_4['recipient_lng'] = data_4['recipient_lng'] / scaling_factor
data_4["grab_lat"] = data_4["grab_lat"] / scaling_factor
data_4["grab_lng"] = data_4["grab_lng"] / scaling_factor

In [175]:
WORK_RESOLUTION = 13  # All courier positions and specific locations (restaurants, customers) are mapped to this grid.
MACRO_RESOLUTION = 8 # Define the lower resolution for strategic demand analysis. res = 8 ~0.74 km² #How many available couriers in the grid versus whats the demand
"""
Convert all relevant lat/lon coordinates from the DataFrame into H3 cell indices.
This maps every point of interest (restaurants, customers, courier starting points) onto our discrete hexagon grid.
"""
try:
    data_4['sender_h3'] = data_4.apply(
        lambda row: h3.latlng_to_cell(row['sender_lat'], row['sender_lng'], WORK_RESOLUTION),
        axis=1
    )
    data_4['recipient_h3'] = data_4.apply(
        lambda row: h3.latlng_to_cell(row['recipient_lat'], row['recipient_lng'], WORK_RESOLUTION),
        axis=1
    )
    data_4['grab_h3'] = data_4.apply(
        lambda row: h3.latlng_to_cell(row['grab_lat'], row['grab_lng'], WORK_RESOLUTION),
        axis=1
    )
    print("Conversion to H3 indices complete.")


except h3.H3ValueError as e:
    print("ERROR: A coordinate in the dataset is invalid. H3 message: {e}")



Conversion to H3 indices complete.


In [176]:
DISTANCE_CACHE = {}
def get_hex_distance(start_hex, end_hex):
    """
    Berechnet die Distanz mit der schnellen h3.grid_distance Funktion
    UND nutzt einen Cache, um wiederholte Berechnungen zu vermeiden.
    """
    key = tuple(sorted((start_hex, end_hex)))
    if key in DISTANCE_CACHE:
        return DISTANCE_CACHE[key]
    try:
        distance = h3.grid_distance(start_hex, end_hex)
        DISTANCE_CACHE[key] = distance
        return distance
    except (h3.H3FailedError, TypeError):
        return float('inf')

In [177]:
# Wenden Sie Ihre bereits existierende Funktion auf die H3-Spalten an
data_4['hex_distance'] = data_4.apply(
    lambda row: get_hex_distance(row['sender_h3'], row['recipient_h3']),
    axis=1
)

# Zeigen Sie die ersten Zeilen mit der neuen Distanz-Spalte an
print(data_4[['sender_h3', 'recipient_h3', 'hex_distance']].head())

         sender_h3     recipient_h3  hex_distance
0  8d329b5aa0b193f  8d329b585a3033f           296
1  8d329b58826397f  8d329b589d71c3f           137
2  8d329b5aba8bdbf  8d329b588972a3f           524
3  8d329b5aa2c82bf  8d329b58171a33f           517
4  8d329b5aa2c82bf  8d329b5ab3988ff           273


In [178]:
data_4['push_hour'] = pd.to_datetime(data_4['order_push_time'], unit='s').dt.hour

In [238]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# Alte Feature-Liste (Beispiel):
# features_alt = ['is_weekend', 'push_hour', 'recipient_lat', 'recipient_lng', 'sender_lng', 'sender_lat']

# ✅ Neue, verbesserte Feature-Liste:
features_neu = [
    'is_weekend',
    'push_hour',
    #'estimate_meal_prepare_time',
    'hex_distance'  # <-- Wichtigstes neues Feature!
]

# Definieren Sie X und y
X = data_4[features_neu]
y = data_4['is_courier_grabbed'] # oder Ihre Zielvariable für die Ablehnung

# Daten aufteilen
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# ✅ Modell neu trainieren
# Dieses Modell "versteht" jetzt den Zusammenhang zwischen Distanz und Ablehnung
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [239]:
import joblib

# Ihr trainiertes Modell-Objekt
# model = LogisticRegression(class_weight='balanced')
# model.fit(X_train, y_train)

# Speichern Sie das trainierte Modell in einer Datei
model_filename = 'rejection_model.joblib'
joblib.dump(model, model_filename)

print(f"Modell wurde erfolgreich in '{model_filename}' gespeichert.")

Modell wurde erfolgreich in 'rejection_model.joblib' gespeichert.


In [232]:
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Modellgenauigkeit: {accuracy:.2f}")

Modellgenauigkeit: 0.86


In [ ]:
import pandas as pd
import h3

# Stellen Sie sicher, dass Ihr trainiertes `model` und die `get_hex_distance` Funktion
# im Speicher verfügbar sind, wenn Sie diese Funktion aufrufen.

def predict_rejection_probability(new_order_dict, resolution=13):
    """
    Nimmt die Daten einer neuen Bestellung, berechnet die Hexagon-Distanz
    und gibt die Ablehnungswahrscheinlichkeit vom trainierten Modell zurück.
    """
    # 1. Konvertiere die Koordinaten der neuen Bestellung in H3-Zellen
    sender_h3 = h3.latlng_to_cell(new_order_dict['sender_lat'],new_order_dict['sender_lng'], resolution)
    recipient_h3 = h3.latlng_to_cell(new_order_dict['recipient_lat'], new_order_dict['recipient_lng'],resolution)

    # 2. Berechne die Hexagon-Distanz
    hex_dist = get_hex_distance(sender_h3, recipient_h3)
    print(hex_dist)

    # 3. Bereite die Features für das Modell vor (genau wie beim Training)
    features_for_prediction = {
        'is_weekend': new_order_dict['is_weekend'],
        #'push_hour': new_order_dict['push_hour'],
        #'estimate_meal_prepare_time': new_order_dict['estimate_meal_prepare_time'],
        'hex_distance': hex_dist
    }

    # 4. Erstelle einen DataFrame mit einer Zeile für die Vorhersage
    # Die Spaltenreihenfolge MUSS mit der beim Training übereinstimmen!
    df_for_prediction = pd.DataFrame([features_for_prediction])

    # 5. Mache die Vorhersage
    # model.predict_proba gibt Wahrscheinlichkeiten für [Klasse 0, Klasse 1] zurück.
    # Wir wollen die Wahrscheinlichkeit für die Ablehnung (angenommen Klasse 1).
    rejection_prob = model.predict_proba(df_for_prediction)[:, 0]

    return rejection_prob[0]

In [234]:
# --- Anwendung ---

# Beispiel für eine neue Bestellung (mit den alten Koordinaten)
new_order = {
    'is_weekend': 1,
    #'estimate_meal_prepare_time': 1667859800,
    'push_hour': 9,
    'recipient_lat': 45.987722,
    'recipient_lng': 174.555740,
    "sender_lng": 174.555740,
    "sender_lat": 45.887722
}

# Vorhersage für die neue Bestellung mit der aktualisierten Funktion
rejection_prob = predict_rejection_probability(new_order)

print(f"\nDie geschätzte Wahrscheinlichkeit, dass diese Bestellung abgelehnt wird, beträgt: {rejection_prob:.2%}")

1705

Die geschätzte Wahrscheinlichkeit, dass diese Bestellung abgelehnt wird, beträgt: 17.84%
